In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import SplineTransformer


def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

# Sample quarterly data
# data = {'quarter': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
#         'value': [10, 15, 12, 18, 11, 16, 13, 19, 12, 17, 14, 20]}
# df = pd.DataFrame(data)

# # Create periodic spline transformer
# qt = periodic_spline_transformer(12, 4)

# # Transform the 'quarter' feature
# quarter = qt.fit_transform(df_train[['Month']])[:,0]

# # Add transformed features to DataFrame
# print(quarter)

In [ ]:
check_pg=[]

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("365D") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    check_pg.append(freqencies)
    check_pg.append(spectrum)
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    plt.savefig('pg.png')
    return ax

In [ ]:
!pip install optuna-integration[catboost]
!pip install -U holidays

In [ ]:
!pip install -U pycountry

In [ ]:
import statsmodels

In [ ]:
import pandas as pd
import numpy as np
import optuna
import catboost as cb
import matplotlib
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from optuna.integration import CatBoostPruningCallback
import pycountry 
import holidays




In [ ]:
df_train=pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
df_test=pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')



In [ ]:
df_train.info()

In [ ]:
df_train['store'].unique()

In [ ]:
df_train[df_train['country']=='Kenya']['store'].unique()

In [ ]:
df_train['product'].unique()

In [ ]:
### import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Generate synthetic time series data
np.random.seed(42)
#df_train['date']=pd.to_datetime(df_train['date'])
#df_train=df_train.set_index('date')
df=df_train['num_sold']

#time = pd.date_range(start=df_train['date'].min, end=df_train['date'].max, periods=365, freq='D')
#series = 10 + 0.05 * np.arange(100) + np.sin(2 * np.pi * time.dayofyear / 365) + np.random.normal(0, 1, 100)
#df = pd.DataFrame({'date': time, 'value': series}).set_index('date')

# Decompose time series
decomposition = seasonal_decompose(
    df_train[(df_train['country']=='Singapore') & 
    (df_train['date'].str.contains('2013')) &
    (df_train['store']=='Premium Sticker Mart')]['num_sold'],
    model='multiplicative', period=30)  # Set period according to data frequency

# Plot components
decomposition.plot()
plt.show()


In [ ]:
# temp=pd.concat([df_train['date'],df_train['num_sold']],axis=1)
# sumsale2016=temp.groupby('date').mean().squeeze().loc['2013']
# #sumsale2016
# plot_periodogram(sumsale2016)

In [ ]:
df_train.info()

In [ ]:
# Plot showing timeseries attributes: trend, seasonality 

# Create subplots: 2 rows, 3 columns
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Flatten the axes array for easy iteration
axes = axes.flatten()
productlist=df_train['product'].unique().tolist()
# Loop through each subplot and plot
for i, ax in enumerate(axes):
    if i<len(productlist):
        df=df_train[(df_train['product']==productlist[i])][['date','num_sold']]
        df.dropna(inplace=True)
        df.set_index('date',inplace=True)
        df['rolling_mean'] = df['num_sold'].rolling(window=30).mean()
        df['rolling_std'] = df['num_sold'].rolling(window=30).std()
        df[['num_sold', 'rolling_mean', 'rolling_std']].plot(ax=ax,title=f'{productlist[i]} Rolling Mean and Std Dev')

plt.tight_layout()

# Show the plot
plt.show()
    

In [ ]:


def get_alpha2_code(country_name):
  """
  Gets the ISO 3166-1 Alpha-2 country code for the given country name.

  Args:
    country_name: The name of the country.

  Returns:
    The ISO 3166-1 Alpha-2 country code, or None if the country is not found.
  """
  try:
    country = pycountry.countries.search_fuzzy(country_name)[0]
    return country.alpha_2
  except LookupError:
    return None



In [ ]:
# Function to create holiday sets for each country
def generate_country_holidays(countries, start_year, end_year):
    country_holidays = {}
    for country in countries:
        # Generate a holiday set for the range of years
        country_holidays[country] = holidays.CountryHoliday(country, years=range(start_year, end_year + 1)) 
    return country_holidays


In [ ]:
df_train['Day']=pd.to_datetime(df_train['date']).dt.strftime("%d")
df_train['Month']=pd.to_datetime(df_train['date']).dt.strftime("%m")
df_train['Year']=pd.to_datetime(df_train['date']).dt.strftime("%Y")
df_train['Day_of_week']=pd.to_datetime(df_train['date']).dt.dayofweek
df_train['Week_No']=pd.to_datetime(df_train['date']).dt.isocalendar().week
df_train['is_weekend']=(df_train['Day_of_week']>=4).astype(int)
df_train['Quarter_Discrete'] = (df_train['Month'].astype(int) - 1) // 3 + 1
df_train['quarter_normalized'] = (df_train['Quarter_Discrete']-1)/3
df_train['month_normalized']=(df_train['Month'].astype(int)-1)/11
df_train['Month_Spline']=periodic_spline_transformer(12,4).fit_transform(df_train[['quarter_normalized']])[:,0]
df_train['Quarter_Spline']=periodic_spline_transformer(4,4).fit_transform(df_train[['quarter_normalized']])[:,0]
df_train['week_number_normalized'] = df_train['Week_No'] / df_train['Week_No'].max()
df_train['Week_Spline']=periodic_spline_transformer(53,5).fit_transform(df_train[['week_number_normalized']])[:,0]

In [ ]:
def consecutive_days_visited(df):
    """
    Calculates the number of consecutive days a store was visited.

    Args:
        df: pandas DataFrame with columns 'date', 'country', 'store'

    Returns:
        pandas DataFrame with an additional column 'consecutive_days' 
    """

    # Ensure date index is sorted
    df = df.sort_index()

    # Group by 'country' and 'store'
    grouped = df.groupby(['country', 'store'])

    # Calculate consecutive days within each group
    df['consecutive_days'] = (grouped.indices - grouped.indices.min()).dt.days + 1

    return df


In [ ]:
allcountries=df_train['country'].unique()
countrycodes = list(map(lambda x:get_alpha2_code(x),allcountries)) 
unique_countries = [cc for cc in countrycodes if cc is not None]
print(countrycodes)

In [ ]:
df_train['store']=df_train['store'].astype('category')
df_train['country']=df_train['country'].astype('category')
df_train['product']=df_train['product'].astype('category')
df_train['Day']=df_train['Day'].astype('category')
df_train['Month']=df_train['Month'].astype('category')
df_train['Year']=df_train['Year'].astype('category')
df_train['Day_of_week']=df_train['Day_of_week'].astype('category')


In [ ]:
df_test['Day']=pd.to_datetime(df_test['date']).dt.strftime("%d")
df_test['Month']=pd.to_datetime(df_test['date']).dt.strftime("%m")
df_test['Year']=pd.to_datetime(df_test['date']).dt.strftime("%Y")
df_test['Day_of_week']=pd.to_datetime(df_test['date']).dt.dayofweek
df_test['Week_No']=pd.to_datetime(df_test['date']).dt.isocalendar().week
df_test['is_weekend']=(df_test['Day_of_week']>=4).astype(int)
df_test['Quarter_Discrete'] = (df_test['Month'].astype(int) - 1) // 3 + 1
df_test['quarter_normalized'] = (df_test['Quarter_Discrete']-1)/3
df_test['month_normalized']=(df_test['Month'].astype(int)-1)/11
df_test['Month_Spline']=periodic_spline_transformer(12,4).fit_transform(df_test[['quarter_normalized']])[:,0]
df_test['Quarter_Spline']=periodic_spline_transformer(4,4).fit_transform(df_test[['quarter_normalized']])[:,0]
df_test['week_number_normalized'] = df_test['Week_No'] / df_test['Week_No'].max()
df_test['Week_Spline']=periodic_spline_transformer(53,5).fit_transform(df_test[['week_number_normalized']])[:,0]


In [ ]:
df_test.drop(['week_number_normalized','month_normalized','quarter_normalized'],axis=1,inplace=True)

In [ ]:
df_train.drop(['week_number_normalized','month_normalized','quarter_normalized'],axis=1,inplace=True)

In [ ]:
df_test['store']=df_test['store'].astype('category')
df_test['country']=df_test['country'].astype('category')
df_test['product']=df_test['product'].astype('category')
df_test['Day']=df_test['Day'].astype('category')
df_test['Month']=df_test['Month'].astype('category')
df_test['Year']=df_test['Year'].astype('category')
df_test['Day_of_week']=df_test['Day_of_week'].astype('category')


In [ ]:
df_train['date']=pd.to_datetime(df_train['date'])
df_test['date']=pd.to_datetime(df_test['date'])

In [ ]:
df_train['previous_visit']=df_train.reset_index().groupby(by=['country','store'])[['date']].diff()

In [ ]:
df_train['consec_days']=df_train.groupby(by=['country','store'])[['previous_visit']].cumsum()

In [ ]:
df_test['previous_visit']=df_test.reset_index().groupby(by=['country','store'])[['date']].diff()
df_test['consec_days']=df_test.groupby(by=['country','store'])[['previous_visit']].cumsum()

In [ ]:
df_train.set_index('date',inplace=True)
Startmin=df_train.index.min()
df_train['days_since_start'] = (df_train.index - Startmin).days

# Add cycle hints based on time 
df_train['sin_year'] = np.sin(2 * np.pi * df_train['days_since_start'] / 365)
df_train['cos_year'] = np.cos(2 * np.pi * df_train['days_since_start'] / 365)
df_train['sin_month'] = np.sin(2 * np.pi * df_train['days_since_start'] / 30)
df_train['cos_month'] = np.cos(2 * np.pi * df_train['days_since_start'] / 30)

df_test.set_index('date',inplace=True)
df_test['days_since_start'] = (df_test.index - Startmin).days

# Add cycle hints based on time 
df_test['sin_year'] = np.sin(2 * np.pi * df_test['days_since_start'] / 365)
df_test['cos_year'] = np.cos(2 * np.pi * df_test['days_since_start'] / 365)
df_test['sin_month'] = np.sin(2 * np.pi * df_test['days_since_start'] / 30)
df_test['cos_month'] = np.cos(2 * np.pi * df_test['days_since_start'] / 30)

df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [ ]:
df_train.info()

In [ ]:
#Get country

#Get holidays for each country
#allcountries.append("weirdcountry")
country_holidays={}

#df_train['date']=pd.to_datetime(df_train['date'])
start_year = df_train['date'].dt.year.min()
end_year = df_train['date'].dt.year.max()

country_holidays = generate_country_holidays(unique_countries, start_year, end_year)
holiday_df = pd.DataFrame([
    {'date': date, 'country_code': country, 'country': cname}
    for country in unique_countries
    for date in country_holidays[country]
    for cname in [pycountry.countries.get(alpha_2=cc).name for cc in unique_countries]]
)
holiday_df['is_holiday'] = 1
holiday_df['date']=pd.to_datetime(holiday_df['date'])
holiday_df.drop('country_code',axis=1,inplace=True)

df_train = df_train.merge(holiday_df, on=['date', 'country'], how='left').fillna({'is_holiday': 0})

In [ ]:
df_train[(df_train['country']=='Norway') & (df_train['store']=='Discount Stickers') & (df_train['previous_visit'].astype('int')>0)].consec_days.max

In [ ]:
start_year = df_test['date'].dt.year.min()
end_year = df_test['date'].dt.year.max()
#unique_countries = [cc for cc in countrycodes if cc is not None]
country_holidays = generate_country_holidays(unique_countries, start_year, end_year)

holiday_df = pd.DataFrame([
    {'date': date, 'country_code': country, 'country': cname}
    for country in unique_countries
    for date in country_holidays[country]
    for cname in [pycountry.countries.get(alpha_2=cc).name for cc in unique_countries]]
)
holiday_df['is_holiday']=1
holiday_df['date']=pd.to_datetime(holiday_df['date'])
holiday_df.drop('country_code',axis=1,inplace=True)

df_test = df_test.merge(holiday_df, on=['date', 'country'], how='left').fillna({'is_holiday': 0})


In [ ]:
df_test['is_holiday']=df_test['is_holiday'].astype(int)
df_test['is_holiday'].value_counts()

In [ ]:
df_train.set_index('date',inplace=True)
#df_test.set_index('date',inplace=True)

In [ ]:
#Graph for intrpolate is changing too much
#Not interpolate but just drop Na is better
#Interpolate missing num_sold

#df_train['date']=pd.to_datetime(df_train['date'])
#df = df_train.set_index('date')

# df_train = df_train.sort_values(by=['country', 'store', 'product', 'date'])

# # Impute NaN with interpolation (grouped by Country, Store, and Product)
# df_train['num_sold'] = df_train.groupby(['country', 'store', 'product'])['num_sold'].transform(
#     lambda group: group.interpolate(method='linear',limit=None,limit_direction='both').ffill().bfill()
# )
#df['num_sold'].isna().any()

In [ ]:
#condition = (df.num_sold.notna()) & (df['country']=='Canada') & (df['store']=='Discount Stickers') & (df['product']=='Holographic Goose')
#df[condition]


In [ ]:
##Get num_sold NaN product store and country
#t=df[df.num_sold.isna()].groupby(['country','store','product'])['product'].value_counts().to_dict()

In [ ]:
#Country store and product that have NaN
# for  k,v in t.items():
#     if v!=0:
#         print(k,v)

In [ ]:
#Find Holographic Goose num_sold in each country
nan_product = df_train[(df_train['product']=='Holographic Goose') & (df_train['num_sold'].notna())].groupby(['country'])['num_sold']
print("Mean")
print(nan_product.mean())
print("Median")
print(nan_product.median())
print("Mode")
print(nan_product.agg(pd.Series.mode))
print("Min")
print(nan_product.min())
print("Max")
print(nan_product.max())

Impute Canana NaN with 218.0 
Impute Kenya NaN with 5.0

In [ ]:
df_train['previous_visit'].fillna(pd.to_timedelta(0),inplace=True)
df_train['consec_days'].fillna(pd.to_timedelta(0),inplace=True)
df_test['previous_visit'].fillna(pd.to_timedelta(0),inplace=True)
df_test['consec_days'].fillna(pd.to_timedelta(0),inplace=True)

df_train.dropna(inplace=True)
#df_test.dropna(inplace=True)

In [ ]:
#Impute Canada with 218
df_train.loc[df_train['country']=='Canada',['num_sold']]=df_train.loc[df_train['country']=='Canada',['num_sold']].fillna(218.0)

In [ ]:
#Impute Kenya with 5
df_train.loc[df_train['country']=='Kenya',['num_sold']]=df_train.loc[df_train['country']=='Kenya',['num_sold']].fillna(5.0)

In [ ]:
# nsmode = df_train['num_sold'].mode()
# df_train['num_sold']=df_train['num_sold'].fillna(nsmode[0])

In [ ]:
#Get date back to feature after interpolation
df_train=df_train.reset_index()

In [ ]:
df_train.info()

In [ ]:
df_train['is_holiday']=df_train['is_holiday'].astype('int')
df_train['previous_visit']=pd.to_numeric(df_train['previous_visit'].dt.days,errors='coerce')
df_train['consec_days']=pd.to_numeric(df_train['consec_days'].dt.days,errors='coerce')
df_train['country']=df_train['country'].astype('category')

df_test['is_holiday']=df_test['is_holiday'].astype('int')
df_test['previous_visit']=pd.to_numeric(df_test['previous_visit'].dt.days,errors='coerce')
df_test['consec_days']=pd.to_numeric(df_test['consec_days'].dt.days,errors='coerce')
df_test['country']=df_test['country'].astype('category')

In [ ]:
#log transform target
df_train['num_sold']=np.log1p(df_train['num_sold']) 


In [ ]:
#Split data according to product into
from catboost import Pool
cv_product_train_pool={}
product_train_pool={}
product_valid_pool={}


for p in productlist:
    df=df_train[df_train['product']==p]
    train_x, valid_x, train_y, valid_y = train_test_split(df.drop(['id','num_sold','product'],axis=1),df['num_sold'],shuffle=False,test_size=0.20)
    cvtrain_x, cvvalid_x, cvtrain_y, cvvalid_y = train_test_split(df.drop(['id','num_sold','product'],axis=1),df['num_sold'],shuffle=False,test_size=0.10) 
    cats=df.select_dtypes('category').columns.to_list()
    cats.remove('product')
    cats.append('is_holiday')
    cats.append('Week_No')
    cats.append('is_weekend')
    cats.append('Quarter_Discrete')
    #train_weights = np.linspace(0.1, 1.0, len(train_x))
    #valid_weights = np.linspace(0.1, 1.0, len(valid_x)) # Linearly increasing weights
    # decay_rate=0.75
    # train_weights = decay_rate ** np.arange(len(train_x))[::-1]  # Exponentially increasing weights
    # valid_weights = decay_rate ** np.arange(len(valid_x))[::-1]  # Exponentially increasing weights
    cv_product_train_pool[p]=Pool(cvtrain_x,label=cvtrain_y,cat_features=cats)
    product_train_pool[p]=Pool(train_x,label=train_y,cat_features=cats)#,weight=train_weights)
    product_valid_pool[p]=Pool(valid_x,label=valid_y,cat_features=cats)#,weight=valid_weights)
    print(f"{p} train date {train_x['date'].min()} to {train_x['date'].max()}")
    print(f"{p} train loc  {train_x.index.min()} to {train_x.index.max()}")
    print(f"{p} valid date {valid_x['date'].min()} to {valid_x['date'].max()}") 
    print(f"{p} valid loc {valid_x.index.min()} to {valid_x.index.max()}")

In [ ]:
productlist

In [ ]:
from functools import partial
from catboost import cv

def objective(trial: optuna.Trial, product) -> float:
    #data, target = load_breast_cancer(return_X_y=True)
    #train_x, valid_x, train_y, valid_y = train_test_split(df_train.drop(['id','num_sold','date'],axis=1), df_train['num_sold'], test_size=0.25)
    #cats=df_train.select_dtypes('category').columns
    
    param = {
        "iterations":trial.suggest_int("iterations",1000,7000),
        "learning_rate":trial.suggest_float("learning_rate",0.006,0.03,log=True),
        "objective": trial.suggest_categorical("objective", ["MAE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.7, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "MAE",
        "cat_features": cats,
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

   
    #use catboost cv with timeseries data
    cv_data = cb.cv(
        cv_product_train_pool[p],
        params=param,
        shuffle=False,
        type='TimeSeries',
        fold_count=3,
        early_stopping_rounds=300,
        verbose=0,
    )
    mape = cv_data['test-MAE-mean'].min()
    best_iter = cv_data['test-MAE-mean'].values.argmin()
    # gbm = cb.CatBoostRegressor(**param)

    # pruning_callback = CatBoostPruningCallback(trial, "MAPE")
    # gbm.fit(
    #    # train_x,
    #    # train_y,
    #    # eval_set=[(valid_x, valid_y)],
    #     country_train_pool[c],
    #     eval_set=country_valid_pool[c],
    #     verbose=0,
    #     early_stopping_rounds=300,
    #     callbacks=[pruning_callback],
    # )

    # evoke pruning manually.
    #pruning_callback.check_pruned()

    
    #preds = gbm.predict(valid_x)
    #pred_labels = np.fix(preds)
    #mape = mean_absolute_percentage_error(valid_y, pred_labels)
    return mape


In [ ]:
from functools import partial 

product_best_params={}
for p in productlist:
    objective = partial(objective, product = p)
    
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    study = optuna.create_study(
        #pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
        direction="minimize",
    )
    study.optimize(objective, n_trials=100, timeout=600)
    
    print("Number of finished trials: {}".format(len(study.trials)))
    
    print(f'{p} Best trial:')
    trial = study.best_trial
    
    print("  Value: {}".format(trial.value))
    
    print(f"{p}  Params: ")
    product_best_params[p]=trial.params.copy()
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        

In [ ]:
#print(study.best_params)

In [ ]:
# from catboost import Pool,cv
# cats=df_train.select_dtypes('category').columns
# features=df_train.drop(['id','num_sold','date'],axis=1).columns
# train_pool = Pool(data=df_train.drop(['id','num_sold','date'],axis=1),
#              label=df_train['num_sold'],
#              cat_features=cats.to_list())

In [ ]:
# from catboost import Pool,cv

#              #feature_names=features)


# # params = {"iterations": 1000,
# #           "depth": 4,
# #           "loss_function": "MAPE",
# #           "verbose": False}

# scores = cv(train_pool,
#             study.best_params,
#            fold_count=5,
#             verbose=0,
#            )

# print(scores)

In [ ]:
#print(scores)

In [ ]:
#hyperdict=optuna.importance.get_param_importances(study)

In [ ]:
#print(hyperdict)

In [ ]:
for p,v in product_best_params.items():
    print(p)
    print(v)

In [ ]:
#Execute this cell in case of loading fitted model to do prediction
product_model={}
def LoadModel():
    Path='/kaggle/input/cbfitproductmodels'
    productlist=['Holographic Goose','Kaggle','Kaggle Tiers','Kerneler','Kerneler Dark Mode']
    for p in productlist:
        product_model[p]=cb.CatBoostRegressor().load_model(f'{Path}/cbfit_{"".join(p.split())}')


In [ ]:
# product_best_params['Holographic Goose']={'iterations': 6363, 'learning_rate': 0.008332793554259138, 'objective': 'MAPE', 'colsample_bylevel': 0.07433491028736375, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}
# product_best_params['Kaggle']={'iterations': 6305, 'learning_rate': 0.010009455831247686, 'objective': 'MAPE', 'colsample_bylevel': 0.03233257672712704, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.298102836318726}
# product_best_params['Kaggle Tiers']={'iterations': 6762, 'learning_rate': 0.018757617260743983, 'objective': 'MAPE', 'colsample_bylevel': 0.028100915871494884, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9219213068275253}
# product_best_params['Kerneler']={'iterations': 6517, 'learning_rate': 0.007996172223793776, 'objective': 'MAPE', 'colsample_bylevel': 0.4791473847946537, 'depth': 5, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.8909693600971766}
# product_best_params['Kerneler Dark Mode']={'iterations': 4739, 'learning_rate': 0.010804008641365494, 'objective': 'MAPE', 'colsample_bylevel': 0.2431153461626964, 'depth': 2, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}

In [ ]:
#Train model with best param and iteration>1000 before prediction
product_model={}
for p in productlist:
    product_model[p]=cb.CatBoostRegressor(**product_best_params[p],verbose=0)
    product_model[p].fit(product_train_pool[p],eval_set=product_valid_pool[p])
    product_model[p].save_model(f'cbfit_{"".join(p.split())}')
    print(f'{p} {product_model[p].get_best_score()}')

In [ ]:
#Use when using model without training
#LoadModel()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
testproductlist=df_test['product'].unique().tolist()
product_predicts={}
for i,p in enumerate(testproductlist):
    df=df_test[df_test['product']==p]
    product_predicts[p] = np.expm1(product_model[p].predict(df.drop(['id','product'],axis=1)))
    df['num_sold']=product_predicts[p]
    df.set_index('date')
    df.loc[:,'rolling_mean'] = df['num_sold'].rolling(window=30).mean()
    df.loc[:,'rolling_std'] = df['num_sold'].rolling(window=30).std()
    df[['num_sold', 'rolling_mean', 'rolling_std']].plot(ax=axes.flat[i],title=f'{p} Rolling Mean and Std Dev')

plt.tight_layout()
plt.show()

In [ ]:
print(product_predicts)

In [ ]:
cdf={}
for p in testproductlist:
    cid=df_test[df_test['product']==p]['id']
    cdf[p]=pd.concat([cid.reset_index(drop=True),pd.DataFrame(np.round(product_predicts[p],0),columns=['num_sold'])],axis=1)
   

In [ ]:
temp = pd.concat([df.reset_index(drop=True) for df in cdf.values()])

In [ ]:
temp=temp.drop_duplicates()

In [ ]:
temp.to_csv('submission_5product_impute_mea_spline_round.csv',index=False)

In [ ]:
cdf={}
for p in testproductlist:
    cid=df_test[df_test['product']==p]['id']
    cdf[p]=pd.concat([cid.reset_index(drop=True),pd.DataFrame(np.fix(product_predicts[p]),columns=['num_sold'])],axis=1)

In [ ]:
#pd.concat([df_test['id'],pd.DataFrame(np.round(predicts,0),columns=['num_sold'])],axis=1).reset_index(drop=True).to_csv('submission_impute_dropna_holidays_conseddays.csv',index=False)
temp = pd.concat([df.reset_index(drop=True) for df in cdf.values()])
temp=temp.drop_duplicates()
temp.to_csv('submission_5product_impute_mea_spline_fix.csv',index=False)